In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

In [ ]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager
from nemo.utils import logging

import os
import wget
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

# Task Description
**Joint Intent and Slot classification** - is a task of classifying an Intent and detecting all relevant Slots (Entities)
for this Intent in a query.
For example, in the query:  `What is the weather in Santa Clara tomorrow morning?`, we would like to classify the query
as a `weather` Intent, and detect `Santa Clara` as a `location` slot and `tomorrow morning` as a `date_time` slot.
Intents and Slots names are usually task specific and defined as labels in the training data.
This is a fundamental step that is executed in any task-driven Conversational Assistant.

Our Bert based model implementation enables to train and then detect both of these tasks together.


# Dataset and NeMo data format

In this tutorial we are going to use a virtual assistant interaction data set that can be downloaded from here: https://github.com/xliuhw/NLU-Evaluation-Data.
There are about 10K training and 1K testing queries which cover 64 various Intents and 55 Slots. 

To work with NeMo NLP classification model, this dataset should be first converted to the NeMo format, which requires next files:
- **dict.intents.csv** - list of all intent names in the data. One line per an intent name.
- **dict.slots.csv** - list of all slot names in the data. One line per a slot name. It is possible to use both: B- I- notations, for separating between first and intermediate tokens for multi token slots. Or just use one slot type for each token of multi token slot. Our recommendation is to use later one, since it is simpler and there is no visible degradation in performance.
- **train.tsv/test.tsv** - contain original queries, one per line, and intent number separated by tab. For example: `what alarms do i have set right now	0`. Intent numbers are according to the intent line in the intent dictionary file (dict.intents.csv) starting from 0. First line of these files contains a header line: `sentence \tab label`.
- **train_slot.tvs/test_slot.tsv** - contain one line per a query, where instead each token there is a number of the token from the slots dictionary file (dict.slots.csv), starting from 0. Last 'out-of scope' token is usually located in the last line of the dictionary. Example: `54 0 0 54 54 12 12` (numbers separated by space). No header line in these files.

NeMo provides **import_dataset.py** converter for few reference datasets (Assistant / Atis / Snips) which converts them to the NeMo data format for the Intent and Slot classification model. If you have your own annotated dataset in a different format, you will need to write a data converter. Possible recommended format for your own annotation, is to have one text file per all examples of one intent. With one line per query in a form like: `did i set an alarm to [alarm_type : wake up] in the [timeofday : morning]`, using brackets to define slot names. This is very similar to the assistant format from this example and you can use its converter to NeMo format with small changes. 

You can run this utility as follows:

**python examples/nlp/intent_slot_classification/data/import_datasets.py --dataset_name=assistant --source_data_dir=source_dir_name --target_data_dir=target_dir_name**


# Download, preprocess and explore the dataset
## Download the dataset and convert it to the NeMo format

In [ ]:
# you can replace DATA_DIR and NEMO_DIR with your own locations
DATA_DIR = "."
NEMO_DIR = '.'

# download the converter files from github for the purpose of this tutorial
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/import_datasets.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/assistant_utils.py', NEMO_DIR)

In [ ]:
# download and unzip the example dataset from github
print('Downloading dataset...')
wget.download('https://github.com/xliuhw/NLU-Evaluation-Data/archive/master.zip', DATA_DIR)
! unzip {DATA_DIR}/NLU-Evaluation-Data-master.zip -d {DATA_DIR}

In [ ]:
# convert the dataset to the NeMo format
!python {NEMO_DIR}/import_datasets.py --dataset_name=assistant --source_data_dir={DATA_DIR}/NLU-Evaluation-Data-master --target_data_dir={DATA_DIR}/nemo_format


## Data exploration
You can see the dataset in both the original and NeMo's formats. We have here 65 different Intents and 55 Slots, which could be typical commands for virtual assistants. Out of scope slot has the name 'O' and is the last in the dictionary of Slots. And we can see examples of queries and also format of training intent and slot files. 

In [ ]:
# list of queries divided by intent files in the original training dataset
! ls -l {DATA_DIR}/NLU-Evaluation-Data-master/dataset/trainset

In [ ]:
# print all intents from the NeMo format intent dictionary
!echo 'Intents: ' $(wc -l < {DATA_DIR}/nemo_format/dict.intents.csv)
! cat {DATA_DIR}/nemo_format/dict.intents.csv

In [ ]:
# print all slots from the NeMo format slot dictionary
!echo 'Slots: ' $(wc -l < {DATA_DIR}/nemo_format/dict.slots.csv)
! cat {DATA_DIR}/nemo_format/dict.slots.csv

In [ ]:
# examples from the intent training file
! head -n 10 {DATA_DIR}/nemo_format/train.tsv

In [ ]:
# examples from the slot training file
! head -n 10 {DATA_DIR}/nemo_format/train_slots.tsv

# Training model

## Model configuration

Our Joint Intent and Slot classification model is comprised of the pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model with an Intent and Slot Classification layer on top of it.

All model and training parameters are defined in the **intent_slot_classification_config.yaml** config file. This file is located in the folder **examples/nlp/intent_slot_classification/conf/**. It contains 2 main sections:
- **model**: All arguments that are related to the Model - language model, token classifier, optimizer and schedulers, datasets and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

We will download the config file from repository for the purpose of the tutorial. If you have a version of NeMo installed locally, you can use it from the above folder.

In [ ]:
# download the model config file from repository for the purpose of this example
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/conf/intent_slot_classification_config.yaml', NEMO_DIR)

# print content of the config file
config_file = "intent_slot_classification_config.yaml"
print(config_file)
config = OmegaConf.load(config_file)
print(OmegaConf.to_yaml(config))

## Setting up Data within the config

Among other things, the config file contains dictionaries called train_ds and validation_ds. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

The converter utility creates both training and evaluation files in the same directory, so we need to specify `model.data_dir` parameter to this directory. Also notice that some config lines, including `model.data_dir`, have `???` in place of paths, this means that values for these fields are required to be specified by the user.

`config.model.intent_loss_weight` parameter - is a balance of training loss between Intent and Slot losses, a number between 0 to 1. Its default value is 0.6 which gives slightly higher priority to the Intent loss and it empirically works quite well. You can experiment with this value if you like.
Also you can try to change `config.model.class_balancing` parameter to `weighted_loss` and see if you get better accuracy.

Let's now add the data directory path to the config.

In [ ]:
config.model.data_dir = f'{DATA_DIR}/nemo_format'

## Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem. `config.trainer.max_epochs` - param defines number of training epochs. Usually 50-100 epochs or less should be enough to train on your data. Let's instantiate the Trainer object.

In [ ]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda

config.trainer.precision = 16 if torch.cuda.is_available() else 32

# for mixed precision training, uncomment the line below (precision should be set to 16 and amp_level to O1):
# config.trainer.amp_level = O1

# remove distributed training flags
config.trainer.accelerator = None

# setup a small number of epochs for demonstration purposes of this tutorial
config.trainer.max_epochs = 5

trainer = pl.Trainer(**config.trainer)

## Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it. Model check points during training will be saved in this directory. 

In [ ]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
# the exp_dir provides a path to the current experiment for easy access
print(str(exp_dir))

## Initializing the model and Training

Initial statistics of the dataset will be displayed at the beginning of the training and then Intent and Slot classification report will be displayed after each training epoch.

In [ ]:
# initialize the model
model = nemo_nlp.models.IntentSlotClassificationModel(config.model, trainer=trainer)

# train
trainer.fit(model)

After training for 5 epochs, which should take no more than few minutes, you can expect training precision for this data set to be around these numbers (the accuracy will gradually continue to improve for this dataset up to about 50 epochs of training): 
```
Intents:
    label                                                precision    recall       f1           support   
    alarm_query (label_id: 0)                               94.74      94.74      94.74         19
    alarm_remove (label_id: 1)                             100.00     100.00     100.00         11
    alarm_set (label_id: 2)                                 85.71      94.74      90.00         19
    audio_volume_down (label_id: 3)                          0.00       0.00       0.00          8
    audio_volume_mute (label_id: 4)                        100.00      86.67      92.86         15
    audio_volume_up (label_id: 5)                           56.52     100.00      72.22         13
    calendar_query (label_id: 6)                            55.00      57.89      56.41         19
    calendar_remove (label_id: 7)                           88.89      84.21      86.49         19
    calendar_set (label_id: 8)                              81.25      68.42      74.29         19
    cooking_recipe (label_id: 9)                            86.36     100.00      92.68         19
    datetime_convert (label_id: 10)                          0.00       0.00       0.00          8
    datetime_query (label_id: 11)                           65.52     100.00      79.17         19
    email_addcontact (label_id: 12)                        100.00      12.50      22.22          8
    email_query (label_id: 13)                              83.33      78.95      81.08         19
    email_querycontact (label_id: 14)                       62.50      78.95      69.77         19
    email_sendemail (label_id: 15)                          70.83      89.47      79.07         19
    general_affirm (label_id: 16)                           95.00     100.00      97.44         19
    general_commandstop (label_id: 17)                     100.00     100.00     100.00         19
    general_confirm (label_id: 18)                         100.00     100.00     100.00         19
    general_dontcare (label_id: 19)                        100.00     100.00     100.00         19
    general_explain (label_id: 20)                         100.00      94.74      97.30         19
    general_joke (label_id: 21)                            100.00     100.00     100.00         12
    general_negate (label_id: 22)                           95.00     100.00      97.44         19
    general_praise (label_id: 23)                          100.00      94.74      97.30         19
    general_quirky (label_id: 24)                           40.00      10.53      16.67         19
    general_repeat (label_id: 25)                          100.00     100.00     100.00         19
    iot_cleaning (label_id: 26)                             84.21     100.00      91.43         16
    iot_coffee (label_id: 27)                               94.74      94.74      94.74         19
    iot_hue_lightchange (label_id: 28)                      94.44      89.47      91.89         19
    iot_hue_lightdim (label_id: 29)                        100.00      83.33      90.91         12
    iot_hue_lightoff (label_id: 30)                         89.47      89.47      89.47         19
    iot_hue_lighton (label_id: 31)                           0.00       0.00       0.00          3
    iot_hue_lightup (label_id: 32)                          81.25      92.86      86.67         14
    iot_wemo_off (label_id: 33)                             60.00     100.00      75.00          9
    iot_wemo_on (label_id: 34)                             100.00      14.29      25.00          7
    lists_createoradd (label_id: 35)                        78.95      78.95      78.95         19
    lists_query (label_id: 36)                              78.95      78.95      78.95         19
    lists_remove (label_id: 37)                             90.00      94.74      92.31         19
    music_likeness (label_id: 38)                           70.59      66.67      68.57         18
    music_query (label_id: 39)                              77.78      73.68      75.68         19
    music_settings (label_id: 40)                            0.00       0.00       0.00          7
    news_query (label_id: 41)                               77.78      73.68      75.68         19
    play_audiobook (label_id: 42)                           90.00      94.74      92.31         19
    play_game (label_id: 43)                                80.00      84.21      82.05         19
    play_music (label_id: 44)                               53.85      73.68      62.22         19
    play_podcasts (label_id: 45)                            89.47      89.47      89.47         19
    play_radio (label_id: 46)                               93.75      78.95      85.71         19
    qa_currency (label_id: 47)                              95.00     100.00      97.44         19
    qa_definition (label_id: 48)                            85.00      89.47      87.18         19
    qa_factoid (label_id: 49)                               45.16      73.68      56.00         19
    qa_maths (label_id: 50)                                100.00     100.00     100.00         14
    qa_stock (label_id: 51)                                 95.00     100.00      97.44         19
    recommendation_events (label_id: 52)                    94.44      89.47      91.89         19
    recommendation_locations (label_id: 53)                 94.74      94.74      94.74         19
    recommendation_movies (label_id: 54)                   100.00     100.00     100.00         10
    social_post (label_id: 55)                              90.00      94.74      92.31         19
    social_query (label_id: 56)                             94.74     100.00      97.30         18
    takeaway_order (label_id: 57)                           93.75      78.95      85.71         19
    takeaway_query (label_id: 58)                           85.71      94.74      90.00         19
    transport_query (label_id: 59)                          83.33      78.95      81.08         19
    transport_taxi (label_id: 60)                          100.00     100.00     100.00         18
    transport_ticket (label_id: 61)                         89.47      89.47      89.47         19
    transport_traffic (label_id: 62)                       100.00     100.00     100.00         19
    weather_query (label_id: 63)                           100.00      89.47      94.44         19
    -------------------
    micro avg                                               85.04      85.04      85.04       1076
    macro avg                                               81.13      80.81      79.36       1076
    weighted avg                                            84.10      85.04      83.54       1076
    
Slots:
    label                                                precision    recall       f1           support   
    alarm_type (label_id: 0)                                 0.00       0.00       0.00          0
    app_name (label_id: 1)                                   0.00       0.00       0.00          6
    artist_name (label_id: 2)                                0.00       0.00       0.00         21
    audiobook_author (label_id: 3)                           0.00       0.00       0.00          1
    audiobook_name (label_id: 4)                             0.00       0.00       0.00         18
    business_name (label_id: 5)                             60.00      56.60      58.25         53
    business_type (label_id: 6)                              0.00       0.00       0.00         24
    change_amount (label_id: 7)                              0.00       0.00       0.00         25
    coffee_type (label_id: 8)                                0.00       0.00       0.00          4
    color_type (label_id: 9)                                 0.00       0.00       0.00         12
    cooking_type (label_id: 10)                              0.00       0.00       0.00          0
    currency_name (label_id: 11)                            84.09      75.51      79.57         49
    date (label_id: 12)                                     57.95      91.07      70.83        112
    definition_word (label_id: 13)                           0.00       0.00       0.00         20
    device_type (label_id: 14)                              74.55      51.25      60.74         80
    drink_type (label_id: 15)                                0.00       0.00       0.00          0
    email_address (label_id: 16)                             0.00       0.00       0.00         14
    email_folder (label_id: 17)                              0.00       0.00       0.00          1
    event_name (label_id: 18)                              100.00      13.24      23.38         68
    food_type (label_id: 19)                                51.72      69.77      59.41         43
    game_name (label_id: 20)                                60.00      14.29      23.08         21
    game_type (label_id: 21)                                 0.00       0.00       0.00          0
    general_frequency (label_id: 22)                         0.00       0.00       0.00          9
    house_place (label_id: 23)                              93.33      42.42      58.33         33
    ingredient (label_id: 24)                                0.00       0.00       0.00          6
    joke_type (label_id: 25)                                 0.00       0.00       0.00          4
    list_name (label_id: 26)                                 0.00       0.00       0.00         21
    meal_type (label_id: 27)                                 0.00       0.00       0.00          0
    media_type (label_id: 28)                                0.00       0.00       0.00         37
    movie_name (label_id: 29)                                0.00       0.00       0.00          0
    movie_type (label_id: 30)                                0.00       0.00       0.00          0
    music_album (label_id: 31)                               0.00       0.00       0.00          0
    music_descriptor (label_id: 32)                          0.00       0.00       0.00          3
    music_genre (label_id: 33)                               0.00       0.00       0.00          9
    news_topic (label_id: 34)                                0.00       0.00       0.00         17
    order_type (label_id: 35)                                0.00       0.00       0.00         17
    person (label_id: 36)                                   44.86      92.31      60.38         52
    personal_info (label_id: 37)                             0.00       0.00       0.00         20
    place_name (label_id: 38)                               71.25      77.03      74.03        148
    player_setting (label_id: 39)                            0.00       0.00       0.00          1
    playlist_name (label_id: 40)                             0.00       0.00       0.00          1
    podcast_descriptor (label_id: 41)                        0.00       0.00       0.00         13
    podcast_name (label_id: 42)                              0.00       0.00       0.00          4
    radio_name (label_id: 43)                               66.67      10.53      18.18         38
    relation (label_id: 44)                                  0.00       0.00       0.00         17
    song_name (label_id: 45)                                 0.00       0.00       0.00         22
    time (label_id: 46)                                     70.27      78.20      74.02        133
    time_zone (label_id: 47)                                 0.00       0.00       0.00          9
    timeofday (label_id: 48)                                 0.00       0.00       0.00         28
    transport_agency (label_id: 49)                          0.00       0.00       0.00          9
    transport_descriptor (label_id: 50)                      0.00       0.00       0.00          0
    transport_name (label_id: 51)                            0.00       0.00       0.00          4
    transport_type (label_id: 52)                           78.38      82.86      80.56         35
    weather_descriptor (label_id: 53)                        0.00       0.00       0.00         17
    O (label_id: 54)                                        92.42      98.80      95.50       5920
    -------------------
    micro avg                                               89.10      89.10      89.10       7199
    macro avg                                               21.86      18.56      18.18       7199
    weighted avg                                            84.42      89.10      86.01       7199
```

## Evaluation
To see how the model performs, we can evaluate the performance of the trained model on a test data file. Here we would load the best checkpoint (the one with the lowest validation loss) and create a model (eval_model) from the checkpoint. We will use the same trainer for testing.

In [ ]:
# extract the path of the best checkpoint from the training, you may update it to any other saved checkpoint file
checkpoint_path = trainer.checkpoint_callback.best_model_path

# load the model from this checkpoint
eval_model = nemo_nlp.models.IntentSlotClassificationModel.load_from_checkpoint(checkpoint_path=checkpoint_path)

In [ ]:
# we will setup testing data reusing the same config (test section)
eval_model.setup_test_data(test_data_config=config.model.test_ds)

# run the evaluation on the test dataset
trainer.test(model=eval_model, ckpt_path=None, verbose=False)

## Inference from Examples
Next step to see how the trained model will classify Intents and Slots for given queries from this domain. To improve the predictions you may need to train the model for more than 5 epochs.


In [ ]:
queries = [
    'set alarm for seven thirty am',
    'lower volume by fifty percent',
    'what is my schedule for tomorrow',
]

pred_intents, pred_slots = eval_model.predict_from_examples(queries, config.model.test_ds)

logging.info('The prediction results of some sample queries with the trained model:')
for query, intent, slots in zip(queries, pred_intents, pred_slots):
    logging.info(f'Query : {query}')
    logging.info(f'Predicted Intent: {intent}')
    logging.info(f'Predicted Slots: {slots}')

## Training Script

If you have NeMo installed locally (eg. cloned from the Github), you can also train the model with the example script: `examples/nlp/intent_slot_classification/intent_slot_classification.py.`
This script contains an example on how to train, evaluate and perform inference with the IntentSlotClassificationModel.

To run a training script, use:

`cd examples/nlp/intent_slot_classification`

`python intent_slot_classification.py model.data_dir=PATH_TO_DATA_DIR`

By default, this script uses examples/nlp/intent_slot_classification/conf/intent_slot_classification_config.py config file, and you may update all the params inside of this config file or alternatively providing them in the command line.

# Multi-Label Intent Classification
---

Similar to the previous task, instead of classifying only one intent for an utterance, we have a model to train for multiple intents. For this tutorial, we will be using the ATIS dataset. 

## Download the dataset and convert it to the NeMo format

In [ ]:
# download the converter files from github for the purpose of this tutorial
DATA_DIR = "/home/rchen/datafolder"
NEMO_DIR = '/home/rchen/nemofolder'
BRANCH = 'main'


files = [f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.dict.intent.csv', 
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.dict.slots.csv',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.dict.vocab.csv',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.test.intent.csv',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.test.pkl', 
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.test.query.csv',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.test.slots.csv', 
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.train.intent.csv',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.train.pkl',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.train.query.csv',
         f'https://raw.githubusercontent.com/howl-anderson/ATIS_dataset/master/data/raw_data/ms-cntk-atis/atis.train.slots.csv']

         
for file in files:
    wget.download(file, DATA_DIR)


# download the converter files from github for the purpose of this tutorial
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/import_datasets.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/assistant_utils.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/chenrichard10/NeMo/{BRANCH}/examples/nlp/multi_label_intent_slot_classification/data/convert_datasets.py', NEMO_DIR)

# Get original atis dataset
!python {NEMO_DIR}/import_datasets.py --dataset_name=atis --source_data_dir={DATA_DIR} --target_data_dir={DATA_DIR}/nemo_format
# Script will create new files at {DATA_DIR}/new_format
!mkdir {DATA_DIR}/new_format
!python {NEMO_DIR}/convert_datasets.py --source_data_dir={DATA_DIR}/nemo_format --target_data_dir={DATA_DIR}/new_format


DATA_DIR = f"{DATA_DIR}/new_format"

## Training the Model

In [1]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager
from nemo.utils import logging

import os
import wget
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

In [8]:
# download the model config file from repository for the purpose of this example
wget.download(f'https://raw.githubusercontent.com/chenrichard10/NeMo/{BRANCH}/examples/nlp/multi_label_intent_slot_classification/conf/multi_label_intent_slot_classification_config.yaml', NEMO_DIR)

# print content of the config file
config_file = f"{NEMO_DIR}/multi-label-intent-slot-classification.yaml"
print(config_file)
config = OmegaConf.load(config_file)
print(OmegaConf.to_yaml(config))

100% [................................................................................] 3259 / 3259/home/rchen/nemofolder/multi-label-intent-slot-classification.yaml
trainer:
  gpus: 2
  num_nodes: 1
  max_epochs: 5
  max_steps: null
  accumulate_grad_batches: 1
  precision: 32
  accelerator: ddp
  log_every_n_steps: 1
  val_check_interval: 1.0
  resume_from_checkpoint: null
  checkpoint_callback: false
  logger: false
model:
  nemo_path: null
  data_dir: ???
  class_labels:
    intent_labels_file: intent_labels.csv
    slot_labels_file: slot_labels.csv
  class_balancing: null
  intent_loss_weight: 0.6
  pad_label: -1
  ignore_extra_tokens: false
  ignore_start_end: true
  train_ds:
    prefix: train
    batch_size: 32
    shuffle: true
    num_samples: -1
    num_workers: 8
    drop_last: false
    pin_memory: false
  validation_ds:
    prefix: test
    batch_size: 32
    shuffle: false
    num_samples: -1
    num_workers: 8
    drop_last: false
    pin_memory: false
  test_ds:
    pr

In [6]:
import time

config.model.data_dir = f"{DATA_DIR}/new_format"
config.model.validation_ds.prefix = "dev"
config.model.test_ds.prefix = "dev"
config.model.class_balancing = "weighted_loss"
config.model.head.num_output_layers = 2
config.trainer.max_epochs = 40
run_name = "test"

# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda

# config.trainer.precision = 16 if torch.cuda.is_available() else 32
config.trainer.precision = 32

# remove distributed training flags
config.trainer.accelerator = None

trainer = pl.Trainer(**config.trainer)
config.exp_manager.exp_dir = os.path.join(DATA_DIR, "output/" + run_name)
config.exp_manager.create_checkpoint_callback = True
config.exp_manager.version = time.strftime('%Y-%m-%d_%H-%M-%S')

exp_dir = exp_manager(trainer, config.get("exp_manager", None))
model = nemo_nlp.models.MultiLabelIntentSlotClassificationModel(config.model, trainer=trainer)
trainer.fit(model)

SyntaxError: EOL while scanning string literal (3312526059.py, line 3)

## Evaluation

To see how the model performs, we can evaluate the performance of the trained model on a test data file. Here we would load the best checkpoint (the one with the lowest validation loss) and create a model (eval_model) from the checkpoint. We will use the same trainer for testing. We will need to use the file specified in the output during the training process.

In [ ]:
# specify checkpoint path with .nemo file (Look for /MultiLabelIntentSlot.nemo)
checkpoint_path = "/home/rchen/datafolder/new_format/output/test/MultiLabelIntentSlot/2022-02-14_08-52-53/checkpoints/MultiLabelIntentSlot.nemo"

# load the model from this checkpoint
eval_model =  nemo_nlp.models.MultiLabelIntentSlotClassificationModel.restore_from(checkpoint_path)

### Optimizing Threshold
Before using our model, we find the probability rounding threshold that gives us the best results on the validation set. 

In [ ]:
eval_model.optimize_threshold(config.model.test_ds)

###  Inference from Examples
Similar to the previous example we can run inference to see how the trained model will classify Intents and Slots for given queries from this domain. To improve the predictions you may need to train the model for more than 10 epochs.


In [ ]:
queries = [
    'i would like to find a flight from charlotte to las vegas that makes a stop in st. louis',
    'on april first i need a ticket from tacoma to san jose departing before 7 am',
    'how much is the limousine service in boston',
]

# We use the optimized threshold for predictions
pred_intents, pred_slots, pred_list = eval_model.predict_from_examples(queries, config.model.test_ds, eval_model.threshold)
logging.info('The prediction results of some sample queries with the trained model:')
    
for query, intent, slots in zip(queries, pred_intents, pred_slots):
    logging.info(f'Query : {query}')
    logging.info(f'Predicted Intents: {intent}')
    logging.info(f'Predicted Slots: {slots}')

### Data Augmentation
---

In scenarios when we don't have many multiple labels, data augmentation can be very useful. This can be done by concatentating utterances together with the word "and" or a period.  A script has been provided below to help with augmentation, but it could be changed depending on your use case.

In [3]:
# download the model config file from repository for the purpose of this example
DATA_DIR = "/home/rchen/datafolder"
NEMO_DIR = '/home/rchen/nemofolder'
BRANCH = 'main'
wget.download(f'https://raw.githubusercontent.com/chenrichard10/NeMo/{BRANCH}/examples/nlp/multi_label_intent_slot_classification/data/augment_training_data.py', NEMO_DIR)

100% [................................................................................] 4296 / 4296

'/home/rchen/nemofolder/augment_training_data (1).py'

In [5]:
!python {NEMO_DIR}/augment_training_data.py --source_data_dir={DATA_DIR}/new_format --target_data_dir={DATA_DIR}/augmented_data --num_mixed=10

In [9]:
import time

config.model.data_dir = f"{DATA_DIR}/augmented_data"
config.model.validation_ds.prefix = "dev"
config.model.test_ds.prefix = "dev"
config.model.class_balancing = "weighted_loss"
config.model.head.num_output_layers = 2
config.trainer.max_epochs = 40
run_name = "test"

# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda

# config.trainer.precision = 16 if torch.cuda.is_available() else 32
config.trainer.precision = 32

# remove distributed training flags
config.trainer.accelerator = None

trainer = pl.Trainer(**config.trainer)
config.exp_manager.exp_dir = os.path.join(DATA_DIR, "output/" + run_name)
config.exp_manager.create_checkpoint_callback = True
config.exp_manager.version = time.strftime('%Y-%m-%d_%H-%M-%S')

exp_dir = exp_manager(trainer, config.get("exp_manager", None))
model = nemo_nlp.models.MultiLabelIntentSlotClassificationModel(config.model, trainer=trainer)
trainer.fit(model)

[NeMo W 2022-02-18 08:36:02 nemo_logging:349] /home/rchen/anaconda3/envs/mynewenv/lib/python3.7/site-packages/pytorch_lightning/loops/epoch/training_epoch_loop.py:49: LightningDeprecationWarning: Setting `max_steps = None` is deprecated in v1.5 and will no longer be supported in v1.7. Use `max_steps = -1` instead.
      "Setting `max_steps = None` is deprecated in v1.5 and will no longer be supported in v1.7."
    
[NeMo W 2022-02-18 08:36:02 nemo_logging:349] /home/rchen/anaconda3/envs/mynewenv/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:148: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
      f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
    
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


[NeMo I 2022-02-18 08:36:02 exp_manager:283] Experiments will be logged at /home/rchen/datafolder/output/test/MultiLabelIntentSlot/2022-02-18_08-36-02


[NeMo W 2022-02-18 08:36:02 exp_manager:880] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to -1. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.
[NeMo W 2022-02-18 08:36:02 nemo_logging:349] /home/rchen/anaconda3/envs/mynewenv/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:244: LightningDeprecationWarning: `ModelCheckpoint(every_n_val_epochs)` is deprecated in v1.4 and will be removed in v1.6. Please use `every_n_epochs` instead.
      "`ModelCheckpoint(every_n_val_epochs)` is deprecated in v1.4 and will be removed in v1.6."
    


[NeMo I 2022-02-18 08:36:02 tokenizer_utils:125] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


...

[NeMo I 2022-02-18 08:51:56 intent_slot_classification_model:296] Intent report: 
    label                                                precision    recall       f1           support   
    abbreviation (label_id: 0)                              71.74     100.00      83.54         33
    aircraft (label_id: 1)                                  40.91     100.00      58.06          9
    airfare (label_id: 2)                                   80.26     100.00      89.05         61
    airline (label_id: 3)                                   76.92     100.00      86.96         40
    airport (label_id: 4)                                   90.00     100.00      94.74         18
    capacity (label_id: 5)                                  87.50     100.00      93.33         21
    cheapest (label_id: 6)                                   0.00       0.00       0.00          0
    city (label_id: 7)                                      36.36      66.67      47.06          6
    day_name (label

Epoch 38, global step 8774: val_loss reached 0.58874 (best 0.58468), saving model to "/home/rchen/datafolder/output/test/MultiLabelIntentSlot/2022-02-18_08-36-02/checkpoints/MultiLabelIntentSlot--val_loss=0.5887-epoch=38.ckpt" as top 3


Validating: 0it [00:00, ?it/s]

[NeMo I 2022-02-18 08:52:20 intent_slot_classification_model:296] Intent report: 
    label                                                precision    recall       f1           support   
    abbreviation (label_id: 0)                              71.74     100.00      83.54         33
    aircraft (label_id: 1)                                  40.91     100.00      58.06          9
    airfare (label_id: 2)                                   80.26     100.00      89.05         61
    airline (label_id: 3)                                   76.92     100.00      86.96         40
    airport (label_id: 4)                                   90.00     100.00      94.74         18
    capacity (label_id: 5)                                  84.00     100.00      91.30         21
    cheapest (label_id: 6)                                   0.00       0.00       0.00          0
    city (label_id: 7)                                      36.36      66.67      47.06          6
    day_name (label

Epoch 39, global step 8999: val_loss was not in top 3
Saving latest checkpoint...


In [10]:
model.optimize_threshold(config.model.test_ds)

[NeMo I 2022-02-18 08:53:44 intent_slot_classification_dataset:92] Setting max length to: 35
[NeMo I 2022-02-18 08:53:44 data_preprocessing:387] Some stats of the lengths of the sequences:
[NeMo I 2022-02-18 08:53:44 data_preprocessing:393] Min: 4 |                  Max: 35 |                  Mean: 12.647256438969764 |                  Median: 12.0
[NeMo I 2022-02-18 08:53:44 data_preprocessing:395] 75 percentile: 15.00
[NeMo I 2022-02-18 08:53:44 data_preprocessing:396] 99 percentile: 24.00
[NeMo I 2022-02-18 08:53:44 intent_slot_classification_dataset:121] 0 are longer than 35
[NeMo I 2022-02-18 08:53:47 multi_label_intent_slot_classification_model:346] Maximum Threshold for F1-Score: 0.6700000000000002, [Precision, Recall, F1-Score]: [0.926595744680851, 0.9592511013215859, 0.9426406926406926]
[NeMo I 2022-02-18 08:53:47 multi_label_intent_slot_classification_model:349] Maximum Threshold for Precision: 0.9500000000000004, [Precision, Recall, F1-Score]: [0.9725609756097561, 0.70264317

In [12]:
queries = [
    'i would like to find a flight from charlotte to las vegas that makes a stop in st. louis',
    'on april first i need a ticket from tacoma to san jose departing before 7 am',
    'how much is the limousine service in boston',
]

# We use the optimized threshold for predictions
pred_intents, pred_slots, pred_list = model.predict_from_examples(queries, config.model.test_ds, model.threshold)
logging.info('The prediction results of some sample queries with the trained model:')
    
for query, intent, slots in zip(queries, pred_intents, pred_slots):
    logging.info(f'Query : {query}')
    logging.info(f'Predicted Intents: {intent}')
    logging.info(f'Predicted Slots: {slots}')

[NeMo I 2022-02-18 08:54:29 intent_slot_classification_dataset:92] Setting max length to: 22
[NeMo I 2022-02-18 08:54:29 data_preprocessing:387] Some stats of the lengths of the sequences:
[NeMo I 2022-02-18 08:54:29 data_preprocessing:393] Min: 10 |                  Max: 22 |                  Mean: 16.666666666666668 |                  Median: 18.0
[NeMo I 2022-02-18 08:54:29 data_preprocessing:395] 75 percentile: 20.00
[NeMo I 2022-02-18 08:54:29 data_preprocessing:396] 99 percentile: 21.92
[NeMo I 2022-02-18 08:54:29 intent_slot_classification_dataset:121] 0 are longer than 22
[NeMo I 2022-02-18 08:54:30 3019254592:9] The prediction results of some sample queries with the trained model:
[NeMo I 2022-02-18 08:54:30 3019254592:12] Query : i would like to find a flight from charlotte to las vegas that makes a stop in st. louis
[NeMo I 2022-02-18 08:54:30 3019254592:13] Predicted Intents: [('flight', 0.95)]
[NeMo I 2022-02-18 08:54:30 3019254592:14] Predicted Slots: O O O O O O O B-time